<a href="https://colab.research.google.com/github/yasminela/AI-ML/blob/main/detection%2C_traching_and_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q ultralytics
!pip install -q deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 58.5 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

cap = cv2.VideoCapture('/content/Car.mp4')

height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

model = YOLO('yolov8l.pt')
tracker = DeepSort(#max_age=30, n_init=3, max_iou_distance=0.7, #embedder='mobilenet', half=True, bgr=True)

line_position = int(height * 0.8)

counted_up = set()
counted_down = set()
previous_positions = {}
vehicle_classes = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']

while True:
    success, img = cap.read()
    if not success:
        break

    results = model(img, conf=0.5, iou=0.5, verbose=False)
    detections = []

    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            w = x2 - x1
            h = y2 - y1

            if w < 20 or h < 20 or w > width * 0.7 or h > height * 0.7:
                continue

            conf = float(box.conf[0])
            cls = int(box.cls[0])
            label = model.names[cls]

            if label in vehicle_classes:
                x1 = max(0, x1)
                y1 = max(0, y1)
                x2 = min(width, x2)
                y2 = min(height, y2)
                detections.append(([x1, y1, x2-x1, y2-y1], conf, label))

    tracks = tracker.update_tracks(detections, frame=img)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        center_x = int((x1 + x2) / 2)
        center_y = y2

        if track_id in previous_positions:
            prev_y = previous_positions[track_id]

            crossed_line = (prev_y < line_position and center_y >= line_position) or \
                          (prev_y > line_position and center_y <= line_position)

            if crossed_line:
                if prev_y > line_position and center_y <= line_position:
                    if track_id not in counted_up:
                        counted_up.add(track_id)
                        vehicle_class = track.get_det_class() if hasattr(track, 'get_det_class') else "vehicle"

                elif prev_y < line_position and center_y >= line_position:
                    if track_id not in counted_down:
                        counted_down.add(track_id)
                        vehicle_class = track.get_det_class() if hasattr(track, 'get_det_class') else "vehicle"

        previous_positions[track_id] = center_y

        if track_id in counted_up:
            box_color = (255, 0, 0)
            status = "UP"
        elif track_id in counted_down:
            box_color = (0, 0, 255)
            status = "DOWN"
        else:
            box_color = (0, 255, 0)
            status = ""

        cv2.rectangle(img, (x1, y1), (x2, y2), box_color, 3)

        vehicle_class = track.get_det_class() if hasattr(track, 'get_det_class') else ""
        label_text = f'ID:{track_id} {vehicle_class}'
        text_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]

        cv2.rectangle(img, (x1, y1-30), (x1+text_size[0]+10, y1), box_color, -1)
        cv2.putText(img, label_text, (x1+5, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.line(img, (0, line_position), (width, line_position), (0, 255, 255), 4)

    cv2.rectangle(img, (10, 10), (300, 150), (0, 0, 0), -1)
    cv2.rectangle(img, (10, 10), (300, 150), (255, 255, 255), 3)

    cv2.putText(img, f'UP:      {len(counted_up)}', (25, 50),
               cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 100, 0), 2)

    cv2.putText(img, f'DOWN:    {len(counted_down)}', (25, 95),
               cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 100, 255), 2)

    cv2.putText(img, f'TOTAL:   {len(counted_up) + len(counted_down)}', (25, 140),
               cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    video.write(img)
cap.release()
video.release()